In [1]:
%load_ext autoreload

In [2]:
# autoreload chaque module spécifié par %aimport
%autoreload 1

%aimport src.kerasCallbacks
%aimport src.results

from IPython.display import display
from os import path
import time

import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import Dense, Input, BatchNormalization, Dropout
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l1, l2

from src import constants
from src.kerasCallbacks import BestWeightsRestorer
import src.results as resultsUtil
import src.runUtil as runUtil

In [3]:
def load_feature_set():

    file_path = path.join(constants.DATA_PATH, 'marsyas_base_split.csv')
    
    ftrs = np.array(pd.read_csv(file_path, header=None).values[:,2:-1])
    lbls = np.array(pd.read_csv(file_path, header=None).values[:,-1])
    
    return ftrs, lbls

def scale_features(ftrs):
    
    standardscaler = StandardScaler()
    ftrs_scld = standardscaler.fit_transform(ftrs)
    
    return ftrs_scld

def one_hot_labels(lbls):
    
    lbls_1d = lbls.reshape(len(lbls), 1)
        
    oh_encoder = OneHotEncoder(sparse=False)
    lbls_oh = oh_encoder.fit_transform(lbls_1d)
    
    return lbls_oh

In [4]:
# Load the features and the labels
raw_ftrs, raw_lbls = load_feature_set()

lbls_oh = one_hot_labels(raw_lbls)
lbl_encoder = LabelEncoder()
lbl_encoded_lbls = lbl_encoder.fit_transform(raw_lbls)

ftrs = scale_features(raw_ftrs)

In [5]:
def create_model(input_shape):
    
    inputs = Input(shape=input_shape, name='mlb_marsyas_input')
    
    layer = Dense(90, activation='relu', name='mlb_marsyas_dense_1')(inputs)
    layer = Dense(85, activation='relu', name='mlb_marsyas_dense_2')(layer)
    
    layer = Dropout(0.10, name='mlb_marsyas_droupout_1')(layer)
    
    layer = Dense(80, activation='relu', name='mlb_marsyas_dense_3')(layer)
    layer = Dense(75, activation='relu', name='mlb_marsyas_dense_4')(layer)
    layer = Dense(70, activation='relu', name='mlb_marsyas_dense_5')(layer)
    
    layer = BatchNormalization(name='mlb_marsyas_batch_norm_1')(layer)
    
    layer = Dense(65, activation='relu', name='mlb_marsyas_dense_6')(layer)
    layer = Dense(60, activation='relu', name='mlb_marsyas_dense_7')(layer)
    
    layer = BatchNormalization(name='mlb_marsyas_batch_norm_2')(layer)
    
    layer = Dense(55, activation='relu', name='mlb_marsyas_dense_8')(layer)
    layer = Dense(50, activation='relu', name='mlb_marsyas_dense_9')(layer)
    
    layer = Dropout(0.10, name='mlb_marsyas_droupout_2')(layer)
    
    layer = Dense(45, activation='relu', name='mlb_marsyas_dense_10')(layer)
    layer = Dense(40, activation='relu', name='mlb_marsyas_dense_11')(layer)
    
    layer = Dropout(0.10, name='mlb_marsyas_droupout_3')(layer)
    
    layer = Dense(30, activation='relu', name='mlb_marsyas_dense_12')(layer)
    
    outputs = Dense(25, activation='softmax', name='mlb_marsyas_output')(layer)
    
    model = Model(inputs=inputs, outputs=outputs)
    
    model.compile(
        optimizer=Adam(lr=0.001), 
        loss='categorical_crossentropy', 
        metrics=[
            CategoricalAccuracy(name='accuracy')
        ]
    )
    
    return model

In [7]:
experiment_number = 0

In [8]:
def train_fold(x_train, y_train, x_val, y_val, run_name, fold=None):

    input_shape = x_train[0].shape
    model = create_model(input_shape)
    
    if fold is not None:
        run_name += '_fold' + str(fold)
        print('Training fold ' + str(fold))
    
    tk_board = TensorBoard(log_dir=constants.LOGS_PATH + run_name)
    weight_restorer = BestWeightsRestorer(monitor='val_loss', mode='min', verbose=1)
    
    history = model.fit(
        x=x_train,
        y=y_train,
        validation_data=[x_val, y_val],
        epochs=110,
        batch_size=2000,
        shuffle=True,
        callbacks=[tk_board, weight_restorer],
        verbose=1
    )
    
    prob_predictions = model.predict(x_val)
    predictions = prob_predictions.argmax(axis=1)
    
    return {
        'history': history.history,
        'predictions': predictions
    }

experiment_number += 1
run_name = 'new_base_mlp_marsyas_cv_' + str(experiment_number)

results = resultsUtil.cross_validate(
    x=ftrs,
    y=lbls_oh,
    y_label_encoded=lbl_encoded_lbls,
    n_splits=5,
    train_func=train_fold,
    run_name=run_name
)

runUtil.save_run_results(run_name, results)

resultsdf = pd.DataFrame([results]).transpose()

display(resultsdf)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Training fold 1
Train on 114904 samples, validate on 28740 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/110
114904/114904 [==============================] - 2s 13us/sample - loss: 3.0071 - accuracy: 0.1129 - val_loss: 2.9964 - val_accuracy: 0.1472
Epoch 2/110
114904/114904 [==============================] - 1s 5us/sample - loss: 2.7419 - accuracy: 0.1785 - val_loss: 2.8229 - val_accuracy: 0.2025
Epoch 3/110
114904/114904 [==============================] - 1s 5us/sample - loss: 2.6115 - accuracy: 0.2101 - val_loss: 2.6793 - val_accuracy: 0.2100
Epoch 4/110
114904/114904 [==============================] - 1s 5us/sample - loss: 2.5436 - accuracy: 0.2292 - val_loss: 2.5864 - val_accuracy: 0.2276
Epoch 5/110
114904/114904 [==============================] - 1s 4us/sample - loss: 2.5014 - a

Epoch 102/110
114904/114904 [==============================] - 1s 5us/sample - loss: 2.1267 - accuracy: 0.3415 - val_loss: 2.3165 - val_accuracy: 0.2962
Epoch 103/110
114904/114904 [==============================] - 1s 5us/sample - loss: 2.1239 - accuracy: 0.3432 - val_loss: 2.3103 - val_accuracy: 0.3002
Epoch 104/110
114904/114904 [==============================] - 1s 4us/sample - loss: 2.1220 - accuracy: 0.3428 - val_loss: 2.3181 - val_accuracy: 0.2941
Epoch 105/110
114904/114904 [==============================] - 1s 5us/sample - loss: 2.1263 - accuracy: 0.3418 - val_loss: 2.3295 - val_accuracy: 0.2963
Epoch 106/110
114904/114904 [==============================] - 1s 5us/sample - loss: 2.1225 - accuracy: 0.3431 - val_loss: 2.3195 - val_accuracy: 0.2983
Epoch 107/110
114904/114904 [==============================] - 1s 5us/sample - loss: 2.1196 - accuracy: 0.3433 - val_loss: 2.3162 - val_accuracy: 0.2971
Epoch 108/110
114904/114904 [==============================] - 1s 5us/sample - los

Epoch 99/110
114911/114911 [==============================] - 1s 5us/sample - loss: 2.1309 - accuracy: 0.3422 - val_loss: 2.3104 - val_accuracy: 0.2983
Epoch 100/110
114911/114911 [==============================] - 1s 5us/sample - loss: 2.1262 - accuracy: 0.3428 - val_loss: 2.3191 - val_accuracy: 0.2978
Epoch 101/110
114911/114911 [==============================] - 1s 5us/sample - loss: 2.1258 - accuracy: 0.3434 - val_loss: 2.3206 - val_accuracy: 0.2937
Epoch 102/110
114911/114911 [==============================] - 1s 5us/sample - loss: 2.1280 - accuracy: 0.3429 - val_loss: 2.3186 - val_accuracy: 0.2968
Epoch 103/110
114911/114911 [==============================] - 1s 5us/sample - loss: 2.1270 - accuracy: 0.3430 - val_loss: 2.3148 - val_accuracy: 0.2960
Epoch 104/110
114911/114911 [==============================] - 1s 5us/sample - loss: 2.1223 - accuracy: 0.3448 - val_loss: 2.3309 - val_accuracy: 0.2959
Epoch 105/110
114911/114911 [==============================] - 1s 5us/sample - loss

Epoch 96/110
114914/114914 [==============================] - 1s 5us/sample - loss: 2.1287 - accuracy: 0.3385 - val_loss: 2.3058 - val_accuracy: 0.2974
Epoch 97/110
114914/114914 [==============================] - 1s 5us/sample - loss: 2.1257 - accuracy: 0.3400 - val_loss: 2.3116 - val_accuracy: 0.2977
Epoch 98/110
114914/114914 [==============================] - 1s 5us/sample - loss: 2.1276 - accuracy: 0.3386 - val_loss: 2.3154 - val_accuracy: 0.2985
Epoch 99/110
114914/114914 [==============================] - 1s 5us/sample - loss: 2.1241 - accuracy: 0.3396 - val_loss: 2.3080 - val_accuracy: 0.2984
Epoch 100/110
114914/114914 [==============================] - 1s 4us/sample - loss: 2.1251 - accuracy: 0.3410 - val_loss: 2.3214 - val_accuracy: 0.2961
Epoch 101/110
114914/114914 [==============================] - 1s 5us/sample - loss: 2.1211 - accuracy: 0.3418 - val_loss: 2.3110 - val_accuracy: 0.2982
Epoch 102/110
114914/114914 [==============================] - 1s 5us/sample - loss: 2

Epoch 93/110
114922/114922 [==============================] - 1s 5us/sample - loss: 2.1404 - accuracy: 0.3392 - val_loss: 2.3142 - val_accuracy: 0.2988
Epoch 94/110
114922/114922 [==============================] - 1s 5us/sample - loss: 2.1388 - accuracy: 0.3401 - val_loss: 2.3170 - val_accuracy: 0.3011
Epoch 95/110
114922/114922 [==============================] - 1s 5us/sample - loss: 2.1368 - accuracy: 0.3405 - val_loss: 2.3137 - val_accuracy: 0.2953
Epoch 96/110
114922/114922 [==============================] - 1s 5us/sample - loss: 2.1346 - accuracy: 0.3402 - val_loss: 2.3111 - val_accuracy: 0.2976
Epoch 97/110
114922/114922 [==============================] - 1s 5us/sample - loss: 2.1327 - accuracy: 0.3415 - val_loss: 2.3156 - val_accuracy: 0.2983
Epoch 98/110
114922/114922 [==============================] - 1s 5us/sample - loss: 2.1345 - accuracy: 0.3394 - val_loss: 2.3121 - val_accuracy: 0.2967
Epoch 99/110
114922/114922 [==============================] - 1s 5us/sample - loss: 2.13

Epoch 90/110
114925/114925 [==============================] - 1s 5us/sample - loss: 2.1358 - accuracy: 0.3406 - val_loss: 2.3237 - val_accuracy: 0.2957
Epoch 91/110
114925/114925 [==============================] - 1s 5us/sample - loss: 2.1341 - accuracy: 0.3401 - val_loss: 2.3159 - val_accuracy: 0.2963
Epoch 92/110
114925/114925 [==============================] - 1s 5us/sample - loss: 2.1335 - accuracy: 0.3408 - val_loss: 2.3273 - val_accuracy: 0.2958
Epoch 93/110
114925/114925 [==============================] - 1s 5us/sample - loss: 2.1327 - accuracy: 0.3411 - val_loss: 2.3181 - val_accuracy: 0.2965
Epoch 94/110
114925/114925 [==============================] - 1s 5us/sample - loss: 2.1333 - accuracy: 0.3410 - val_loss: 2.3222 - val_accuracy: 0.2967
Epoch 95/110
114925/114925 [==============================] - 1s 5us/sample - loss: 2.1300 - accuracy: 0.3419 - val_loss: 2.3190 - val_accuracy: 0.2954
Epoch 96/110
114925/114925 [==============================] - 1s 4us/sample - loss: 2.13

,0
avg_val_f1_micro,0.297646
avg_val_f1_macro,0.274868
avg_val_accuracy,0.297646
avg_batch_val_accuracy,0.297646
avg_accuracy,0.325711
avg_val_loss,2.298860
avg_loss,2.188799
train_time,329.797179


In [6]:
def train_final_model(x, y, run_name):
    
    save_path = constants.MODELS_PATH + run_name + '.h5' 
    
    input_shape = x[0].shape
    model = create_model(input_shape)
    
    tk_board = TensorBoard(log_dir=constants.LOGS_PATH + run_name)
    
    model.fit(
        x=x,
        y=y,
        epochs=110,
        batch_size=2000,
        shuffle=True,
        callbacks=[tk_board],
        verbose=1
    )
    
    model.save(save_path)


train_final_model(ftrs, lbls_oh, 'base_mlp_marsyas_final3')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Epoch 1/110
143644/143644 [==============================] - 2s 11us/sample - loss: 2.9934 - accuracy: 0.1244
Epoch 2/110
143644/143644 [==============================] - 1s 6us/sample - loss: 2.6700 - accuracy: 0.1967
Epoch 3/110
143644/143644 [==============================] - 1s 6us/sample - loss: 2.5723 - accuracy: 0.2176
Epoch 4/110
143644/143644 [==============================] - 1s 6us/sample - loss: 2.5252 - accuracy: 0.2315
Epoch 5/110
143644/143644 [==============================] - 1s 6us/sample - loss: 2.4901 - accuracy: 0.2421
Epoch 6/110
143644/143644 [==============================] - 1s 6us/sample - loss: 2.4645 - accuracy: 0.2506
Epoch 7/110
143644/143644 [==============================] - 1s 7us/sample - loss: 2.4398 - accuracy: 0.2566
E

143644/143644 [==============================] - 1s 6us/sample - loss: 2.1761 - accuracy: 0.3300
Epoch 67/110
143644/143644 [==============================] - 1s 6us/sample - loss: 2.1771 - accuracy: 0.3300
Epoch 68/110
143644/143644 [==============================] - 1s 6us/sample - loss: 2.1756 - accuracy: 0.3298
Epoch 69/110
143644/143644 [==============================] - 1s 6us/sample - loss: 2.1741 - accuracy: 0.3300
Epoch 70/110
143644/143644 [==============================] - 1s 6us/sample - loss: 2.1724 - accuracy: 0.3310
Epoch 71/110
143644/143644 [==============================] - 1s 6us/sample - loss: 2.1713 - accuracy: 0.3301
Epoch 72/110
143644/143644 [==============================] - 1s 6us/sample - loss: 2.1695 - accuracy: 0.3307
Epoch 73/110
143644/143644 [==============================] - 1s 6us/sample - loss: 2.1686 - accuracy: 0.3312
Epoch 74/110
143644/143644 [==============================] - 1s 6us/sample - loss: 2.1686 - accuracy: 0.3315
Epoch 75/110
143644/143